In [ ]:
###soft_threshodl

def soft_threshodl(groups,nc,w,mu):
    val =[float(np.sign(w1)*np.max(np.abs(w1)-mu,0)) for w1 in w]
    return np.array(val).reshape((len(val),1))

In [ ]:
####lossh
def lossh(y,yhat,mu):
    r= abs(yhat['RET']-y)
    l= np.zeros(len(r))
    ind = (r>mu)
    l[ind] = 2*mu*r[ind]-mu*mu
    ind = (r<= mu)
    l[ind]  = r[ind]*r[ind]
    return np.mean(l)

In [ ]:
####f_gradh
def f_gradh(w,X,y,mu):
    r = np.matmul(X,w)-y['RET']
    ind0 = np.where(abs(r)<= mu)
    ind0 = pd.Series(np.asarray(ind0)[0])
    ind1 = np.where(r>mu)
    ind1 = pd.Series(np.asarray(ind1)[0])

    indf1 = np.where(r< (-mu))
    indf1 = pd.Series(np.array(indf1)[0])
    grad = np.matmul(np.transpose(X.iloc[ind0,:]),(np.matmul(X.iloc[ind0,:],w)-y.iloc[ind0,0]))+np.matmul(np.transpose(mu*X.iloc[ind1,:]),np.ones(len(ind1)))-np.matmul(np.transpose(mu*X.iloc[indf1,:]),np.ones(len(indf1)))
    return grad

In [ ]:
def proximalH_l(groups,nc,xtest,mtrain,ytest,w,X,y,mu,tol,L,l2,func):
    w= np.asarray(w[0])
    dim = X.shape[0]
    max_iter = 3000
    gamma= 1/L
    l1 = l2

    v= w
    yhatbig1=np.matmul(xtest,w) + mtrain.values
    r20=lossh(yhatbig1,ytest,mu)
    for t in range(max_iter):
        vold=v
        w_perv=w
        w= np.asarray(v-(gamma*f_gradh(v,X,y,mu)))
        mu1=l1*gamma
        w=func(groups,nc,w,mu1)
        w = np.asarray([float(w1) for w1 in w])
        v=w + t/(t+3)*(w-w_perv)
        if ((np.linalg.norm(v-vold)**2) < ((np.linalg.norm(vold)**2)*tol) or np.sum(abs(v-vold))==0):
            break
    return v


In [ ]:
import string
import os
import pandas as pd
import datetime
import numpy as np
import statsmodels.api as sm
import math
from scipy.linalg import svd
#import Functions as Fun
import scipy.linalg as la
import scipy.sparse.linalg as sla

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
###Some parameters
mu=0.2;
tol=1e-10;


In [ ]:
X_all = pd.read_csv('../input/project2/X_cleaned.csv')
Y_all = pd.read_csv('../input/project2/Y_cleaned.csv')
Date_index = pd.read_csv('../input/project2/Date_list.csv')
Date_index['DATE']=pd.to_datetime(Date_index['DATE'])

In [ ]:
####for training
nums= 30
start_dates = [pd.to_datetime('1957-03-01')+pd.offsets.DateOffset(years=x) for x in range(nums) ]

In [ ]:
###Files for storing results
r2_oos_all = np.zeros((10,len(start_dates)))
r2_is_all = np.zeros((10,len(start_dates)))

In [ ]:
for m in range(len(start_dates)):

    start_date = start_dates[m]

    end_date =  start_date + pd.offsets.DateOffset(years=18)

    ind_training = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)

    ytrain = Y_all[ind_training]

    xtrain = X_all[ind_training]

    ####for validation

    start_date =  end_date
    end_date = start_date + pd.offsets.DateOffset(years=12)

    ind_vali = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)

    ytest = Y_all[ind_vali]
    xtest= X_all[ind_vali]

    #####for test  oos

    start_date =  end_date
    end_date = start_date + pd.offsets.DateOffset(years=1)
    ind_test = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
    #print(type(ind_test))
    yoos = Y_all[ind_test]
    xoos= X_all[ind_test]

    #######Monthly Demean %%%
    ytrain_demean= ytrain - np.mean(ytrain);
    ytest_demean=ytest-np.mean(ytest);
    mtrain=np.mean(ytrain);
    mtest=np.mean(ytest);

    sd=np.zeros(len(xtrain.columns)) #dim of sd? sd for each characteristics
    for i in range(len(xtrain.columns)):
        s=np.std(xtrain.iloc[:,[i]])
        colnames = xtrain.columns
        if s.values > 0:
            colname = colnames[i]
            xtrain.loc[:,colname]=xtrain[colname]/s.values
            xtest.loc[:,colname]=xtest.loc[:,colname]/s.values
            xoos.loc[:,colname]=xoos.loc[:,colname]/s.values
            sd[i]=s.values
            
    XX =np.matmul(np.transpose(xtrain),xtrain)
    U,S,V=svd(XX)
    #### singular value decomposition such that XX=U*S*V'
    L=S[0]
    ###disp 'Lasso L = '
    ####disp(L)
    Y=ytrain_demean;
    XY= np.matmul(np.transpose(xtrain),Y)
    groups=0
    nc=0
    #### Start to Train %%%

    clf=sm.OLS(ytrain_demean,xtrain)
    yhatres = clf.fit()
    yhatbig1= yhatres.predict(xoos)+ mtrain.values
    #prediction for oos
    r2_oos=1-sum((yhatbig1-yoos['RET'])**2)/sum((yoos['RET']-mtrain.values)**2) # oos r2
    print('Simple OLS R2 : '+np.str0(r2_oos))

    yhatbig1=yhatres.predict(xtrain) + mtrain.values
    r2_is=1-(yhatbig1-ytrain['RET']).pow(2).sum()/sum(pow(ytrain['RET']-mtrain.values,2))
    b= yhatres.params
    b= pd.DataFrame(b)
    print('Simple OLS R2_is : '+np.str0(r2_is))
    
    b=proximalH_l(groups,nc,xtest,mtrain,ytest,b,xtrain,ytrain_demean,mu,tol,L,0,soft_threshodl)
    b= pd.DataFrame(b)
    yhatbig1=np.matmul(xoos,b)+mtrain.values

    r2_oos=1- np.linalg.norm(yhatbig1[0]-yoos['RET'])**2 / np.linalg.norm((yoos['RET']-mtrain.values))**2
    yhatbig1 = np.matmul(xtrain,b) + mtrain.values
    r2_is = 1 - np.linalg.norm(yhatbig1[0]-ytrain['RET'])**2 / np.linalg.norm((ytrain['RET']-mtrain.values))**2
    print('Simple OLS+H R2 : '+np.str0(r2_oos))

<!--
Simple OLS R2 : -40539.60621616937
Simple OLS R2_is : 0.05992692068397043
Simple OLS+H R2 : -30242.21178949597
Simple OLS R2 : -188.9836658458398
Simple OLS R2_is : 0.050372059340969555
Simple OLS+H R2 : -114.58957720796894
Simple OLS R2 : -111.58100828759846
Simple OLS R2_is : 0.042002408525097845
Simple OLS+H R2 : -92.57396091809078
Simple OLS R2 : -22.08519609879732
Simple OLS R2_is : 0.035386773448858544
Simple OLS+H R2 : -15.703426386013938
Simple OLS R2 : -41.263902062670894
Simple OLS R2_is : 0.033801539744170994
Simple OLS+H R2 : -21.198323170594268
Simple OLS R2 : -25.114695938590124
Simple OLS R2_is : 0.03187309383951875
Simple OLS+H R2 : -18.303536288151648
Simple OLS R2 : -23.36758585682776
Simple OLS R2_is : 0.03199738411043063
Simple OLS+H R2 : -27.231317028541202
Simple OLS R2 : -7.5234804273449996
Simple OLS R2_is : 0.029848001066650087
Simple OLS+H R2 : -6.402041074595246
Simple OLS R2 : -185.72947473187043
Simple OLS R2_is : 0.032619377464253585
Simple OLS+H R2 : -372.5903498843558
Simple OLS R2 : -418.2985337609929
Simple OLS R2_is : 0.029767168806058897
Simple OLS+H R2 : -1043.982504289336
Simple OLS R2 : -514.4222415431627
Simple OLS R2_is : 0.029353324608101805
Simple OLS+H R2 : -877.5598992893431
Simple OLS R2 : -201.64450886949155
Simple OLS R2_is : 0.02543973410388989
Simple OLS+H R2 : -340.9332419506964
Simple OLS R2 : -24.645800433358563
Simple OLS R2_is : 0.024046866571127645
Simple OLS+H R2 : -32.0690328454405
Simple OLS R2 : -1.879638798947925
Simple OLS R2_is : 0.021752564220117176
Simple OLS+H R2 : -1.6149032044680869
Simple OLS R2 : -0.9122911085318941
Simple OLS R2_is : 0.019522972700758867
Simple OLS+H R2 : -0.7577842609907823
Simple OLS R2 : -0.21081651085287456
Simple OLS R2_is : 0.017702159383878646
Simple OLS+H R2 : -0.21921152325192028
Simple OLS R2 : -0.24651187526294516
Simple OLS R2_is : 0.015952456849158247
Simple OLS+H R2 : -0.3048200011243982
Simple OLS R2 : -0.4094741485416231
Simple OLS R2_is : 0.017442371985012084
Simple OLS+H R2 : -0.323291377627583
Simple OLS R2 : -0.12488422499985119
Simple OLS R2_is : 0.016603315015737974
Simple OLS+H R2 : -0.12121027636921333
Simple OLS R2 : -0.07685773999571688
Simple OLS R2_is : 0.014378574187012716
Simple OLS+H R2 : -0.05963294737711089
Simple OLS R2 : -0.10907712414353221
Simple OLS R2_is : 0.014119976599667239
Simple OLS+H R2 : -0.0008035634419458759
Simple OLS R2 : 0.009935279248879048
Simple OLS R2_is : 0.014878190164881788
Simple OLS+H R2 : 0.06279537129331048
Simple OLS R2 : -0.05234527695029745
Simple OLS R2_is : 0.014052822076288995
Simple OLS+H R2 : -0.07209879729539326
Simple OLS R2 : -0.0522577651788938
Simple OLS R2_is : 0.013153846193471153
Simple OLS+H R2 : -0.022776676232572735
Simple OLS R2 : -0.012390428876179582
Simple OLS R2_is : 0.011878778811131196
Simple OLS+H R2 : 0.0051849173598340625
Simple OLS R2 : -0.16849490307570836
Simple OLS R2_is : 0.014550913289206546
Simple OLS+H R2 : -0.048553893803920944
Simple OLS R2 : -0.0032421041061905242
Simple OLS R2_is : 0.012967218629671318
Simple OLS+H R2 : -0.0001294083265670487
Simple OLS R2 : -0.03797821339401497
Simple OLS R2_is : 0.012144107056377473
Simple OLS+H R2 : -0.019847483864450677
Simple OLS R2 : -0.0033692978443886634
Simple OLS R2_is : 0.011694732397120156
Simple OLS+H R2 : 0.020381218252002897
Simple OLS R2 : -0.041904891645949016
Simple OLS R2_is : 0.012809817996217054
Simple OLS+H R2 : -0.029523662772608317

In [ ]:
for m in range(len(start_dates)):

    start_date = start_dates[m]

    end_date =  start_date + pd.offsets.DateOffset(years=18)

    ind_training = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)

    ytrain = Y_all[ind_training]

    xtrain = X_all[ind_training]

    ####for validation

    start_date =  end_date
    end_date = start_date + pd.offsets.DateOffset(years=12)

    ind_vali = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)

    ytest = Y_all[ind_vali]
    xtest= X_all[ind_vali]

    #####for test  oos

    start_date =  end_date
    end_date = start_date + pd.offsets.DateOffset(years=1)
    ind_test = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
    #print(type(ind_test))
    yoos = Y_all[ind_test]
    xoos= X_all[ind_test]

    #######Monthly Demean %%%
    ytrain_demean= ytrain - np.mean(ytrain);
    ytest_demean=ytest-np.mean(ytest);
    mtrain=np.mean(ytrain);
    mtest=np.mean(ytest);

    sd=np.zeros(len(xtrain.columns)) #dim of sd? sd for each characteristics
    for i in range(len(xtrain.columns)):
        s=np.std(xtrain.iloc[:,[i]])
        colnames = xtrain.columns
        if s.values > 0:
            colname = colnames[i]
            xtrain.loc[:,colname]=xtrain[colname]/s.values
            xtest.loc[:,colname]=xtest.loc[:,colname]/s.values
            xoos.loc[:,colname]=xoos.loc[:,colname]/s.values
            sd[i]=s.values
    XX =np.matmul(np.transpose(xtrain),xtrain)
    U,S,V=svd(XX)
    #### singular value decomposition such that XX=U*S*V'
    L=S[0]
    ###disp 'Lasso L = '
    ####disp(L)
    Y=ytrain_demean;
    XY= np.matmul(np.transpose(xtrain),Y)
    
#model 3 PCR
    ne=30;
    #XX=xtrain.'*xtrain;
    pca_val,pca_vec=la.eig(XX)
    p1=pca_vec[:,list(range(ne))]
    Z=np.matmul(xtrain,p1)

    r=np.zeros((3,ne))
    B=np.zeros((xtrain.shape[1],ne))
    Y=ytrain_demean


    for j in range(ne-1):
        j=j+1
        xx=Z[range(j)]
        b = np.matmul(np.matmul(np.linalg.inv(np.matmul(np.transpose(xx),xx)),np.transpose(xx)),Y)
        b=np.matmul(p1[:,range(j)],b)
        yhatbig1=np.matmul(xtest,b)+mtrain.values
        # r[0,j-1] = 1 - (np.sum((yhatbig1[0]-ytest['RET'])**2)/np.sum((ytest['RET']-mtrain.values**2)))
        r[0,j-1]=1-(np.linalg.norm((yhatbig1[0]-ytest['RET'])))**2/(np.linalg.norm((ytest['RET']-mtrain.values)))**2
        yhatbig1=np.matmul(xoos,b)+mtrain.values
        r[1,j-1]=1-(np.linalg.norm((yhatbig1[0]-yoos['RET'])))**2/(np.linalg.norm((yoos['RET']-mtrain.values)))**2
        yhatbig1=np.matmul(xtrain,b)+mtrain.values
        r[2,j-1]=1-(np.linalg.norm((yhatbig1[0]-ytrain['RET'])))**2/(np.linalg.norm((ytrain['RET']-mtrain.values)))**2
        B[:,j-1]= b[0]
    b=np.zeros((xtest.shape[1],1))
    j=ne-1;
    yhatbig1=np.matmul(xtest,b)+mtrain.values
    r[0,j]=1-(np.linalg.norm(yhatbig1[0]-ytest['RET']))**2/np.linalg.norm((ytest['RET']-mtrain.values))**2
    yhatbig1=np.matmul(xoos,b)+mtrain.values
    r[1,j]=1-np.linalg.norm((yhatbig1[0]-yoos['RET']))**2/np.linalg.norm((yoos['RET']-mtrain.values))**2
    yhatbig1=np.matmul(xtrain,b)+mtrain.values
    r[2,j]=1-np.linalg.norm((yhatbig1[0]-ytrain['RET']))**2/np.linalg.norm((ytrain['RET']-mtrain.values))**2
    B[:,j]=b[0]

    r2_oos=r[1,np.argmax(r[0,:])]
    r2_is=r[2,np.argmax(r[0,:])]
    b=pd.DataFrame(B[:,np.argmax(r[0,:])] )
    print('PCR R2 : '+np.str0(r2_oos) )
    print('PCR R2_is : '+np.str0(r2_is) )



<!--
PCR R2 : 0.0
PCR R2_is : 0.0
PCR R2 : -0.00010057186274226027
PCR R2_is : 2.948990589501932e-07
PCR R2 : -0.00017068375870277208
PCR R2_is : 5.216279859943107e-06
PCR R2 : -7.7760400025495e-05
PCR R2_is : 4.284656742958859e-06
PCR R2 : 0.0004260360531215257
PCR R2_is : 5.744965787868317e-06
PCR R2 : 0.0
PCR R2_is : 0.0
PCR R2 : 0.0
PCR R2_is : 0.0
PCR R2 : 0.0
PCR R2_is : 0.0
PCR R2 : 0.0
PCR R2_is : 0.0
PCR R2 : 0.0
PCR R2_is : 0.0
PCR R2 : 0.0
PCR R2_is : 0.0
PCR R2 : 0.0
PCR R2_is : 0.0
PCR R2 : 0.0
PCR R2_is : 0.0
PCR R2 : -0.0001829763536771445
PCR R2_is : 8.527285978443544e-07
PCR R2 : 0.0
PCR R2_is : 0.0
PCR R2 : 8.377036013529349e-06
PCR R2_is : 7.317660255523606e-09
PCR R2 : -0.00038721590503421055
PCR R2_is : 3.023437424864994e-07
PCR R2 : -0.0007104589246602622
PCR R2_is : 3.113739387439196e-07
PCR R2 : -9.469682263896217e-06
PCR R2_is : 1.1011032661212283e-06
PCR R2 : 0.00012129271855365609
PCR R2_is : 4.870533505396679e-07
PCR R2 : 0.0
PCR R2_is : 0.0
PCR R2 : 0.005749884543529871
PCR R2_is : 0.00034316925725030867
PCR R2 : -0.011541065856885968
PCR R2_is : 0.0014358504104761138
PCR R2 : 0.0
PCR R2_is : 0.0
PCR R2 : 0.0021331681020578808
PCR R2_is : 0.0010466153863664118
PCR R2 : 0.0
PCR R2_is : 0.0
PCR R2 : -0.00381184789094946
PCR R2_is : 0.0011736426271864442
PCR R2 : 0.00398613626933475
PCR R2_is : 0.0012266046174207457
PCR R2 : 0.003872932566164611
PCR R2_is : 0.0012621887409830501
PCR R2 : 0.0
PCR R2_is : 0.0